In [7]:
%load_ext autoreload
%autoreload 1
import os

from obspy import UTCDateTime
from obspy.clients.fdsn import Client

from insight.catalog import InsightCatalog
from insight.rotate import rotate_zne
from insight.util import EventDownloader, TraceFile

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [8]:
DATA_DIR = "../data/"
DATALESS_SEED = "ELYSE.dataless"
CATALOG_BASE_URL = "https://www.seis-insight.eu/static/mqs-catalogs/v14/"
CATALOG_STD = "events_extended_multiorigin_v14_2023-01-01.xml"
CATALOG_EXT = "events_mars_extended_multiorigin_v14_2023-01-01.xml"

# https://www.iris.edu/hq/sis/insight
network = "XB"
station = "ELYSE"
location = "02"
min_ang, max_ang = 30, 90
before, after = 10, 120
inv_time = UTCDateTime(2020, 1, 1)  # Only used to get channel metadata
channels = ["BHU", "BHV", "BHW"]

# # RFs
# f0 = 2.0
# itmax = 400
# minderr = 0.001

## Fetch Inventory
Get lat/lon and sensor orientations from Iris

In [9]:
client = Client("IRIS")
insight = client.get_stations(network=network, station=station, location=location, channel="BH?", level="channel")[0]
lat, lon = insight[0].latitude, insight[0].longitude
bh_orientations = {
    ch: insight.get_orientation(f"{network}.{station}.{location}.{ch}", inv_time)
    for ch in channels
}

## Build Catalog

In [10]:
cb = InsightCatalog(CATALOG_BASE_URL + CATALOG_EXT, DATA_DIR)
catalog = cb.parse()
events = catalog.events.set_index("earthquake name")
events

,id,region name,time,quality,mqs_azimuth,mqs_distance,p_arrival,M_w,pp_arrival
earthquake name,,,,,,,,,
S1415a,mqs2022wrzi,Elysium Southwest,2022-11-19T21:53:34.0000Z,B,108.853964,0.000050,2022-11-19 21:56:13.594137+00:00,3.3,NaT
S1337a,mqs2022rbsc,Elysium Southwest,2022-08-31T20:23:46.685686Z,B,108.853964,0.000050,NaT,3.3,NaT
S1237b,mqs2022jvts,Elysium Southwest,2022-05-20T23:37:51.098952Z,D,108.853964,0.000050,NaT,NaN,NaT
S1237a,mqs2022jvst,Elysium Southwest,2022-05-20T23:08:41.115997Z,C,108.853964,0.000050,NaT,2.6,NaT
S1235a,mqs2022jrva,Elysium Southwest,2022-05-18T19:45:44.84616Z,C,108.853964,0.000050,NaT,2.7,NaT
...,...,...,...,...,...,...,...,...,...
S0128a,mqs2019gudd,Elysium Southwest,2019-04-07T09:31:52.0000Z,B,108.853964,0.000050,NaT,2.2,NaT
S0116a,mqs2019fxyu,Elysium Southwest,2019-03-26T06:22:19.90061Z,D,108.853964,0.000050,NaT,NaN,NaT
S0105a,mqs2019fddj,Aeolis Northeast,2019-03-14T20:59:21.050826Z,B,112.000000,32.522022,2019-03-14 21:03:03.600000+00:00,2.9,NaT


## Fetch 'A' Quality Events

In [11]:
st_uvw = {}

events_dl = EventDownloader(DATA_DIR, before=before, after=after)
events = events[events["quality"] == "A"]
events = events.dropna(subset=["p_arrival"])
events

for evt in events.index:
    e = events.loc[evt]
    st_uvw[evt] = events_dl.get_stream(
        network=network,
        station=station,
        location=location,
        channel="BH?",
        est_p_arrival=e.p_arrival,
        evt_id=e.id
    )

events

,id,region name,time,quality,mqs_azimuth,mqs_distance,p_arrival,M_w,pp_arrival
earthquake name,,,,,,,,,
S1222a,mqs2022isne,Aeolis Northeast,2022-05-04T23:23:07.710516Z,A,109.000000,37.014014,2022-05-04 23:27:34+00:00,4.6,NaT
S1133c,mqs2022cixp,Elysium Southeast,2022-02-03T08:04:36.469259Z,A,90.000000,30.006507,2022-02-03 08:08:11.700000+00:00,3.8,NaT
S1102a,mqs2022aceh,Syrtis Major Northwest,2022-01-02T04:27:10.093677Z,A,295.000000,73.309309,2022-01-02 04:35:19.300000+00:00,3.2,NaT
S1094b,mqs2021zdzn,Diacria Southwest,2021-12-24T22:38:02.749173Z,A,40.000000,59.653654,2021-12-24 22:44:48.700000+00:00,4.0,NaT
S1048d,mqs2021vwbn,Elysium Southeast,2021-11-07T22:00:15.254098Z,A,100.000000,30.186186,2021-11-07 22:03:42.700000+00:00,3.6,NaT
S1022a,mqs2021tyvj,Elysium Northeast,2021-10-11T23:14:29.105382Z,A,63.000000,30.725225,2021-10-11 23:18:25.025302+00:00,3.6,NaT
S1015f,mqs2021tkqn,Elysium Southeast,2021-10-04T04:52:29.248537Z,A,93.000000,27.490991,2021-10-04 04:56:00.605894+00:00,2.9,NaT
S0864a,mqs2021indu,Elysium Southeast,2021-05-02T00:57:35.34519Z,A,278.295695,28.748749,2021-05-02 01:00:56.500000+00:00,3.1,NaT
S0820a,mqs2021fjzq,Aeolis Northeast,2021-03-18T14:51:33.869889Z,A,88.000000,30.186186,2021-03-18 14:54:39+00:00,3.3,NaT


## Rotate to ZNE

In [12]:
st_zne = {}

for evt in events.index:
    st_zne[evt] = rotate_zne(st_uvw[evt], bh_orientations)

## Save rotated streams and catalog to disk

Sort by M_w

In [13]:
events.sort_values("M_w", ascending=False).to_parquet(os.path.join(DATA_DIR, "a_quality.parquet"), index=True)
zne_dst_dir = os.path.join(DATA_DIR, "zne")
if not os.path.exists(zne_dst_dir):
    os.mkdir(zne_dst_dir)

for evt in events.index:
    trace_file = TraceFile(network, station, location, events.loc[evt].id, format="MSEED")
    st_zne[evt].write(os.path.join(zne_dst_dir, str(trace_file)), format="MSEED")

/Users/tom/Library/Caches/pypoetry/virtualenvs/insight-notebooks-clKRl_iU-py3.11/lib/python3.11/site-packages/obspy/io/mseed/core.py:770: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


In [14]:
st_zne["S1222a"][0].stats

         network: XB
         station: ELYSE
        location: 02
         channel: BHZ
       starttime: 2022-05-04T23:27:35.204000Z
         endtime: 2022-05-04T23:29:45.154000Z
   sampling_rate: 20.0
           delta: 0.05
            npts: 2600
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'M', 'number_of_records': 24, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 36864})